<a href="https://colab.research.google.com/github/muhozag/sparklearning/blob/recommender/RecommenderSystems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems: Movie Example

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [7]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [8]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [11]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [12]:
training_data, test_data = data.randomSplit([0.75, 0.25])

In [13]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [14]:
model = als.fit(training_data)

In [15]:
model.transform(test_data).show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      2|   1.0|    26|    2.776901|
|      2|   1.0|    12|  -0.8582172|
|      0|   1.0|    22|   1.3907368|
|      2|   2.0|     1|    4.491065|
|      3|   1.0|     1|   0.9713031|
|      0|   1.0|    13|  -0.7122837|
|      0|   1.0|     6|  -1.1551566|
|      0|   1.0|    19|  0.73965013|
|      1|   1.0|    19|   2.3638434|
|      0|   1.0|    15|  -0.6869138|
|      2|   1.0|    15|  -1.7304538|
|      2|   1.0|    17|   3.2258954|
|      4|   1.0|     9|   2.0903714|
|      1|   1.0|     4|   0.0702703|
|      0|   1.0|     8|-0.012443304|
|      2|   1.0|    23|   1.9032135|
|      2|   1.0|    25|  0.11984444|
|      2|   4.0|    21|     5.69049|
|      3|   3.0|    14|   1.8032113|
|      2|   3.0|     0|  -2.9139676|
+-------+------+------+------------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(model.transform(test_data))

In [18]:
rmse

1.8873027672126772

**Test on a new user**

In [19]:
single_user = test_data.filter(test_data['userId']==11).select(['movieId','userId'])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|      9|    11|
|     10|    11|
|     11|    11|
|     16|    11|
|     25|    11|
|     27|    11|
|     30|    11|
|     32|    11|
|     37|    11|
|     45|    11|
|     48|    11|
|     67|    11|
|     72|    11|
|     80|    11|
|     86|    11|
+-------+------+



In [21]:
recommendations = model.transform(single_user)

In [22]:
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     32|    11|  5.4705877|
|     67|    11|   5.382704|
|     27|    11|  4.7116504|
|     48|    11|   3.296233|
|     25|    11|  3.2195861|
|     10|    11|   3.147543|
|     80|    11|  2.0223305|
|     16|    11|  1.9611602|
|     86|    11|  1.6061102|
|     37|    11|   1.531517|
|     45|    11|  1.1802475|
|     72|    11|  1.0706995|
|     30|    11|  0.9875894|
|      6|    11| 0.07045202|
|      9|    11|-0.04733655|
|     11|    11| -1.3082914|
+-------+------+-----------+

